In [ ]:
import pandas as pd
pd.set_option('display.max_rows' , 200)

In [ ]:
company_name = 'tcs'.upper()

In [ ]:
share_price_in = 1000000;

In [ ]:
def calculate_growth(initial , final) :
    if not initial :
        return 1
    return round(((final - initial) / initial) * 100,2)

In [ ]:
def get_share_price_in(price) :
    return round((price / share_price_in),2)

    

In [ ]:
def vertical_analysis(item , divider) :
    return (item/divider)
    

In [ ]:
def round_off_to(num , limit) :
    return round(num,limit)

In [ ]:
def getColumnFormatterbyPercentage(type) :
    columns_perc_formats = {}
    if type == 'income_statement' :
        for column in income_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'balance_sheet' :
        for column in balance_sheet.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'cashflow_statement' :
        for column in cashflow_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    return columns_perc_formats

In [ ]:
styles = [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "150%"),
                              ("color", 'white'),
                              ("background", '#308D46'),
                              ("font-weight", '600'),
                              ("text-transform", 'uppercase')
                             ])]

In [ ]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [ ]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [ ]:
income_statement.rename(columns={company_name+'_income-statement_Annual_As_Originally_Reported' : 'items'} , inplace=True)
balance_sheet.rename(columns={company_name+'_balance-sheet_Annual_As_Originally_Reported' : 'items'} , inplace=True)
cashflow_statement.rename(columns={company_name+'_cash-flow_Annual_As_Originally_Reported' : 'items'} , inplace=True)

In [ ]:
basic_eps_filter = income_statement['items'].eq('Basic EPS')
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [ ]:
WASO_filter = income_statement['items'].eq('Basic Weighted Average Shares Outstanding') | income_statement['items'].eq('Diluted Weighted Average Shares Outstanding') | income_statement['items'].eq('Basic WASO') | income_statement['items'].eq('Diluted WASO')
waso_pos = income_statement[WASO_filter].index

In [ ]:
income_statement.dropna(how='all',subset=income_statement.columns[1:], inplace=True)
balance_sheet.dropna(how='all',subset=balance_sheet.columns[1:], inplace=True)
cashflow_statement.dropna(how='all',subset=cashflow_statement.columns[1:], inplace=True)
income_statement.fillna(0 , inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [ ]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [ ]:
temp_df = temp_df.apply(get_share_price_in)

In [ ]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [ ]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [ ]:
income_statement.loc[1,'items'] = income_statement.loc[1,'items'].strip()
temp_revenue = income_statement.loc[1]
income_statement.loc[1] = income_statement.loc[0]
income_statement.loc[0] = temp_revenue 

In [ ]:
is_highlighted_indexes = income_statement.loc[0:basic_eps_pos - 1,'items'].copy()
is_highlighted_indexes = is_highlighted_indexes.apply(lambda x: x[0] != ' ')
is_highlighted_indexes = is_highlighted_indexes.loc[is_highlighted_indexes].index

In [ ]:
income_statement['items'] = income_statement['items'].apply(lambda x: x.strip())

In [ ]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [ ]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [ ]:
vertical_analysis_is = pd.DataFrame(columns=income_statement.columns) 
vertical_analysis_is['items'] = income_statement.loc[0:basic_eps_pos - 1,'items'].copy()


In [ ]:
total_revenue_filter = income_statement['items'].eq('Total Revenue')
total_revenue = income_statement.loc[total_revenue_filter,start_col:]

In [ ]:
for column in vertical_analysis_is.columns[1:] : 
    vertical_analysis_is[column] = income_statement.loc[0:basic_eps_pos - 1,column].apply(vertical_analysis,divider=total_revenue[column])
    

In [ ]:
#def row_style(row):
#    if row['items'] in vertical_analysis_is.loc[is_highlighted_indexes,'items'].values:
#        return pd.Series('background-color: hsl(47,83%,91%);', row.index)
#    else :
#        return pd.Series('', row.index)
#vertical_analysis_is.style.apply(row_style,axis=1)

In [ ]:
vertical_analysis_is.loc[is_highlighted_indexes].style.set_caption(company_name+ " income statement vertical analysis").set_table_styles(styles).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 